<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup> Douglas Ian Kelley,
<sup>2 </sup>Chantelle Burton, 
<sup>3 </sup>Rhys Whitley,
<sup>1 </sup>Chris Huntingford,
<sup>4 </sup>Ioannis Bistinas, 
<sup>1,5 </sup>Megan Brown, 
<sup>6 </sup>Ning Dong, 
<sup>1 </sup>Toby R. Marthews
<br>
<br>
<br>
<sup>*  </sup> douglas.i.kelley@gmail.com
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>3 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<sup>4 </sup>ATOS Nederland B.V., Amstelveen, The Netherlands
<br>
<sup>5 </sup>School of Physical Sciences, The Open University, Milton Keynes, UK
<br>
<sup>6 </sup>Department of Biological Sciences, Macquarie University, North Ryde, NSW 2109, Australia 
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file). And if we ever put some kind of temperal memory into the model, we'll also have to think again.
</p>
<br>
<b>If you are using new data, you will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries


In [1]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
inPath = "../inputs/amazon_region/"
outPath = "../inputs/amazon_inference_data-2002-MCDBA_obs"


# Data starts in 2002, so 16*12 takes is to 2018.
nmonths_import = 16*12

We had an option for optimizing against fire counts from INPE satalite observations. This has been taken out for now at reviewer requests. Though once we get the science behind optimized fire counts on a firmer footing, we'll probably add it back in. Though we might as well export the data still. Below is a list of INPE salalite sources. Cos each satalite has a different time period, we get a different csv file for each satalite with with drivers, burnt area from MCD45 and GFED4s, and fire count from the names satalite. If you dont want to use any INPE fire counts, you can just set ```satalites = None``` and it'll get skipped over, and you'll have one data file in the end.

In [6]:

satalites = ["TERRA_M__M", "TERRA_M__T", "AQUA_M__M", "AQUA_M__T"]

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [7]:
driver_paths_all = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
driver_names_all = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths_all]

file_table = pd.DataFrame({'filepath': driver_paths_all, 'file_name': driver_names_all})

def checkFilename(driver_name, sat):
    
    if "firecount" in driver_name:
        if sat in driver_name: return True
        return False
    
    return True

def select_files_for_sat(sat = None):
    if sat is not None: test = [checkFilename(driver_name, sat) for driver_name in driver_names_all]
    
    driver_paths = np.array(driver_paths_all)[test]
    driver_names = np.array(driver_names_all)[test]
    
    driver_names = ["firecount" if "firecount" in driver_name else driver_name for driver_name in driver_names]
    return driver_paths, driver_names
    
driver_info = [select_files_for_sat(sat) for sat in satalites]

Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [8]:
def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['variable'][:,:,:])
        if (len(gdata) < nmonths_import): return None
        gdata = gdata[13:nmonths_import]
        gdata[gdata < -9E9] = np.nan
        gflat = gdata.flatten()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [9]:
values = [ [nc_extract(dp) for dp in driver_paths[0]] for driver_paths in driver_info]

Processing: ../inputs/amazon_region/climate/air2001-2019.nc
Processing: ../inputs/amazon_region/climate/emc-2001-2019.nc
Processing: ../inputs/amazon_region/climate/lightning2001-2019.nc
Processing: ../inputs/amazon_region/climate/relative_humidity2001-2019.nc
Processing: ../inputs/amazon_region/climate/tasMax_2001-2019.nc
Processing: ../inputs/amazon_region/climate/precip2001-2019.nc
Processing: ../inputs/amazon_region/climate/seasonDiff-2001-2019.nc
Processing: ../inputs/amazon_region/climate/soilw.0-10cm.gauss.2001-2019.nc
Processing: ../inputs/amazon_region/climate/deep_soilw.10-200cm.gauss.2001-2019.nc
Processing: ../inputs/amazon_region/climate/shallow_soilw.0-10cm.gauss.2001-2019.nc
Processing: ../inputs/amazon_region/vegetation/MaxOverMean_deep_soilw.10-200cm.gauss.2001-2019.nc
Processing: ../inputs/amazon_region/vegetation/nonetreecover-2001-June2018.nc
Processing: ../inputs/amazon_region/vegetation/treecover-2001-June2018.nc
Processing: ../inputs/amazon_region/vegetation/vegc

Turn this into a dataframe for the analysis.

In [10]:
# turn list into a dataframe
def makeDataFrave(value, driver_names):
    df = pd.DataFrame(np.array(value).T, columns=driver_names)
    print(df.info())
    df.dropna(inplace=True)
    return df
fire_df = [makeDataFrave(value, driver_names[1]) for value, driver_names in zip(values, driver_info)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114202 entries, 0 to 114201
Data columns (total 28 columns):
air                          44750 non-null float32
emc                          44750 non-null float32
lightning                    44750 non-null float32
relative_humidity            44750 non-null float32
tasMax_                      44750 non-null float32
precip                       44750 non-null float32
seasonDiff                   44750 non-null float32
soilw                        44750 non-null float32
deep_soilw                   44750 non-null float32
shallow_soilw                44750 non-null float32
MaxOverMean_deep_soilw       44750 non-null float32
nonetreecover                44750 non-null float32
treecover                    44750 non-null float32
vegcover                     44750 non-null float32
MaxOverMean_soilw            44750 non-null float32
MaxOverMean_shallow_soilw    44750 non-null float32
firecount                    44750 non-null float32
burnt

Check that we've built it correctly.

In [11]:
for df in fire_df:
    print(df.head(10))

          air       emc   lightning  relative_humidity     tasMax_    precip  \
25  27.087145  0.205117  117.581161          69.133926  304.157135  0.575250   
26  25.412504  0.186246  126.675491          59.910713  303.414307  0.670358   
27  26.206789  0.175376   41.093647          56.357143  303.635712  0.183623   
28  26.552143  0.181387    1.843172          59.107143  303.478577  0.547380   
29  25.338928  0.216764    5.154491          72.044640  302.510712  0.491154   
46  25.305716  0.265329  100.561661          91.437500  299.939301  3.409420   
47  21.976793  0.258696  349.522064          85.455360  298.296448  1.739765   
48  22.679646  0.213658  156.334686          68.142860  299.046448  0.595429   
49  27.701786  0.170746   46.084488          56.000000  303.946442  0.184299   
50  28.437860  0.160914    6.552771          52.803570  305.500000  0.339156   

    seasonDiff     soilw  deep_soilw  shallow_soilw  ...  cropland  fract_agr  \
25    1.003373  0.277269    0.117291  

Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [12]:
for sat, df in zip(satalites, fire_df):
    outPathi = outPath + '-' + sat + '.csv'
    savepath = os.path.expanduser(outPathi)
    df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>